In [1]:
import numpy as np
import matplotlib.pyplot as plt
import visualization
import os
from gta_math import points_to_homo, ndc_to_view, construct_proj_matrix, view_to_world, construct_view_matrix
from visualization import load_depth, save_pointcloud_csv
import progressbar
from pointcloud_to_voxelmap import pointclouds_to_voxelmap
from joblib import Parallel, delayed
from configparser import ConfigParser
from PIL import Image
import pickle
import json
from voxelmaps import camera_to_pointcloud, load_scene_db_data, get_main_image_name, scene_to_voxelmap, get_main_image

In [2]:
ini_file = "gta-postprocessing.ini"
visualization.multi_page = False
visualization.ini_file = ini_file

base_name = '2018-03-30--04-05-57--208'

conn = visualization.get_connection_pooled()
cur = conn.cursor()

CONFIG = ConfigParser()
CONFIG.read(ini_file)
in_directory = CONFIG["Images"]["Tiff"]
out_directory = CONFIG["Images"]["MlDatasetVoxel"]

In [20]:
scene_id = '0249a724-faa8-40b9-b097-70ffd0eb9607'

cameras = load_scene_db_data(scene_id)

image_name = get_main_image_name(cameras)
voxels, values, map_voxel_size = scene_to_voxelmap(cameras)

In [15]:
cam = get_main_image(cameras)
proj_matrix = cam['proj_matrix']
view_matrix = cam['view_matrix']
width = cam['width']
height = cam['height']


In [16]:
x_min = -1
x_max = 1
y_min = -1
y_max = 1
x_range = 1
y_range = 1
z_meters_min = 1.5
z_meters_max = 50
# z min calc
z_min = proj_matrix @ [1, 1, -z_meters_max, 1]
z_min = z_min[2] / z_min[3]
# z max calc
z_max = proj_matrix @ [1, 1, -z_meters_min, 1]
z_max = z_max[2] / z_max[3]
z_range = 100
x_bin = (x_max - x_min) / x_range
y_bin = (y_max - y_min) / y_range
z_bin = (z_max - z_min) / z_range


In [17]:
X, Y, Z, W = np.mgrid[x_min:x_max:x_bin, y_min:y_max:y_bin, z_min:z_max:z_bin, 1:2:1]  # W is here as homo coordinate
positions = np.vstack([X.ravel(), Y.ravel(), Z.ravel(), W.ravel()])

In [35]:
params = {
    'width': width,
    'height': height,
    'proj_matrix': proj_matrix,
}

points_view = ndc_to_view(positions, proj_matrix)
points_world = view_to_world(points_view, view_matrix)
# here I find corresponding voxels for generated points

w = np.ones((1,voxels.shape[1]))
voxels_homo = np.vstack([voxels, w])
voxels_view = view_matrix @ voxels_homo

In [ ]:
save_pointcloud_csv(vecs_p.T[:, 0:3], '{}/{}.csv'.format(in_directory, base_name))
save_pointcloud_csv(positions.T[:, 0:3], '{}/ndc-{}.csv'.format(in_directory, base_name))

In [27]:
voxels.shape

(3, 136698)

In [24]:
w = np.ones((1,voxels.shape[1]))